# Python helper file/script for MATLAB

This cell sets up the notebook to import numpy, datascience, seaborn, pandas, matplotlib etc.

In [20]:
# Run this cell to set up the notebook.

# These lines import the Numpy, Datascience, pandas modules.
import numpy as np
import pandas as pd
import seaborn as sns
from datascience import *
import random
import matplotlib
import matplotlib.pyplot as plt

# Importing plotting libraries and styles
%matplotlib inline
plt.style.use('fivethirtyeight')

# For Pandas to ignore FutureWarning displays
import warnings
warnings.simplefilter('ignore', FutureWarning)

### The cell given below sets up MATLAB for the notebook
Source: https://sehyoun.com/blog/20180904_using-matlab-with-jupyter-notebook.html

In [21]:
import matlab.engine
import io
import scipy.io
from IPython.core.magic import register_cell_magic
ip = get_ipython()

out = io.StringIO()
err = io.StringIO()

# Setup matlab cell magic #
@register_cell_magic
def matlab_magic(line,cell):
    out.truncate(0)
    out.seek(0)
    err.truncate(0)
    err.truncate(0)
    raw = '''{line}.eval("""{cell}""", nargout=0, stdout=out, stderr=err)'''
    ip.run_cell(raw.format(line=line, cell=cell))
    print(out.getvalue())
    print(err.getvalue())
    
# Starting a MATLAB engine called eng
eng = matlab.engine.start_matlab()

In [22]:
%%matlab_magic eng

t = linspace(0,6*pi,100);
plot(sin(t))
grid on
hold on
plot(cos(t), 'r')

In [23]:
%%matlab_magic eng

x = 5;
disp(x);
x

     5


x =

     5





Calling another MATLAB file from Python script

In [24]:
%%matlab_magic eng

my_array = [10, 20, 30, 40, 50]
avgOfThis = averageFunc(my_array)


my_array =

    10    20    30    40    50


avgOfThis =

    30





**Python to MATLAB**

Source: https://stackoverflow.com/questions/10997254/converting-numpy-arrays-to-matlab-and-vice-versa

In [25]:
x = [1,2,3]
y = "hello"
z = [[1,2,3], [4,5,6]]

scipy.io.savemat('test.mat', dict(x=x, y=y, z=z))

In [26]:
%%matlab_magic eng
load test
x
y
z


x =

  1x3 int64 row vector

   1   2   3


y =

    'hello'


z =

  2x3 int64 matrix

   1   2   3
   4   5   6





**MATLAB to Python**

Source: http://www.blogforbrains.com/blog/2014/9/6/loading-matlab-mat-data-in-python

In [27]:
# Similarly we have
#scipy.io.loadmat("averageFunc.m")
variables = scipy.io.loadmat('data.mat', squeeze_me=True)
variables

{'__header__': b'MATLAB 5.0 MAT-file, Platform: MACI64, Created on: Fri Feb  8 01:20:47 2019',
 '__version__': '1.0',
 '__globals__': [],
 'a': array([[1, 2, 3],
        [4, 5, 6]], dtype=uint8),
 'S': array((array([[ 7,  8,  9],
        [10, 11, 12]], dtype=uint8),), dtype=[('b', 'O')]),
 'M': array([(array([[ 2,  4,  6],
        [ 8, 10, 12]], dtype=uint8),),
        (array([[ 1,  3,  5],
        [ 7,  9, 11]], dtype=uint8),)], dtype=[('c', 'O')])}

In [28]:
a = variables['a'] # array
S = variables['S'] # structure containing an array
M = variables['M'] # array of structures

a

array([[1, 2, 3],
       [4, 5, 6]], dtype=uint8)

In [29]:
S

array((array([[ 7,  8,  9],
       [10, 11, 12]], dtype=uint8),), dtype=[('b', 'O')])

In [30]:
M

array([(array([[ 2,  4,  6],
       [ 8, 10, 12]], dtype=uint8),),
       (array([[ 1,  3,  5],
       [ 7,  9, 11]], dtype=uint8),)], dtype=[('c', 'O')])

In [31]:
# Adds the MMB.m as well as MMBOPT1.m and MMBOPT2.m folders to the MATLAB engine path"
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab/2.3.2', nargout=0)
eng.addpath(r'/Users/Desktop/monetaryPolicy/mmb-gui-mlab/2.3.2/MMB_OPTIONS', nargout=0)

**Important:** The cell below runs MMB.m

In [114]:
eng.MMB(nargout = 0)

### Here we import the data for ALL IRF (Impulse Response Functions) variables

In [38]:
old_df = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "all IRFs Mon. Pol. Shock")

In [113]:
df = old_df.T
new_header = df.iloc[0] # grab the first row for the header
df = df[1:] # take the data less the header row
stripped_headers = [myHeader.strip() for myHeader in np.array(new_header)] # removing trailing whitespaces
df.columns = stripped_headers # set the header row as the df header
df["c_t"] = df.index
df.index = np.arange(0,21,1)
df.index.name = "Period"

# This section rearranges the columns
n = len(list(df.columns.values))
rearranged = [list(df.columns.values)[-1]] + list(df.columns.values)[:n-1]
df = df[rearranged]
df

,c_t,wtilde_t,lambda_zstar_t,m_t,pi_t,x_t,s_t,i_t,h_t,kbar_t1,...,AUX_ENDO_LAG_45_3,AUX_ENDO_LAG_44_1,AUX_ENDO_LAG_44_2,AUX_ENDO_LAG_44_3,AUX_ENDO_LAG_41_1,AUX_ENDO_LAG_41_2,AUX_ENDO_LAG_41_3,AUX_ENDO_LAG_43_1,AUX_ENDO_LAG_43_2,AUX_ENDO_LAG_43_3
Period,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.0484745,-0.00699479,-0.00019142,-0.00640682,0.00640682,-0.617748,0.204864,-0.00435076,-0.00772508,-0.00013688,...,0,0,0,0,0,0,0,0,0,0
2,-0.0359882,-0.0057628,0.00374196,-0.628739,0.00458512,0.587575,-0.0129795,-0.00799692,-0.042797,-0.000384165,...,0,-0.00475476,0,0,1.00723,0,0,0.0256273,1.09089e-15,-7.02819e-18
3,-0.0273793,-0.00484467,0.00441633,-0.0444382,0.00327421,0.00774458,-0.00867639,-0.00999072,-0.0337628,-0.000686397,...,0,-0.0277619,-0.00475476,0,0.00260698,1.00723,0,0.0183405,0.0256273,1.08891e-15
4,-0.0209604,-0.00410564,0.00267624,-0.0389977,0.00230403,0.00584697,-0.00528293,-0.0102847,-0.0264263,-0.00098837,...,-0.00475476,-0.0219998,-0.0277619,-0.00475476,0.0103993,0.00260698,1.00723,0.0130968,0.0183405,0.0256273
5,-0.0157588,-0.00345112,0.00152936,-0.0346907,0.00154005,0.0103063,-0.00460463,-0.00952427,-0.020475,-0.00125692,...,-0.0277619,-0.0173575,-0.0219998,-0.0277619,0.0161765,0.0103993,0.00260698,0.00921613,0.0130968,0.0183405
6,-0.0116265,-0.00288615,0.000758499,-0.0253409,0.000956398,0.00776854,-0.00384489,-0.00816057,-0.0153676,-0.00147411,...,-0.0219998,-0.0136148,-0.0173575,-0.0219998,0.0107477,0.0161765,0.0103993,0.00616021,0.00921613,0.0130968
7,-0.00839697,-0.00240694,0.000250941,-0.018096,0.000523723,0.00581319,-0.00311633,-0.00651357,-0.0111004,-0.00163266,...,-0.0173575,-0.010406,-0.0136148,-0.0173575,0.00690904,0.0107477,0.0161765,0.00382559,0.00616021,0.00921613
8,-0.00591378,-0.00200589,-5.71195e-05,-0.0124966,0.000213718,0.0042884,-0.00246846,-0.00480617,-0.00762111,-0.0017325,...,-0.0136148,-0.00772237,-0.010406,-0.0136148,0.00412512,0.00690904,0.0107477,0.00209489,0.00382559,0.00616021


Todo:
* Pull important variables of the IRF (file 1) using *selective row dataframe read*
* Use the same technique to get the Autocorrelation functions, in sheetname *ACF*

### Here we import the data for 4 IRF (Impulse Response Functions) variables
#### outputgap, inflation, interest, output

In [131]:
irf_4 = pd.read_excel("../mmb-gui-mlab-2.3.2/OUTPUT/results.xls", sheetname = "IRF Mon. Pol. Shock      ")
irf_4 = irf_4.T
irf_headers = irf_4.iloc[0] # grab the first row for the header
irf_4 = irf_4[1:] # take the data less the header row
irf_4_stripped_headers = [myHeader.strip() for myHeader in np.array(irf_headers)] # removing trailing whitespaces
irf_4.columns = irf_4_stripped_headers
modelName = irf_4.columns.values[1]
irf_4

,outputgap,US_ACELm,inflation,US_ACELm,interest,US_ACELm,output,US_ACELm
Unnamed: 1,NaN,0,NaN,0,NaN,0,NaN,0
Unnamed: 2,NaN,0.0311337,NaN,2.86822e-17,NaN,1.01557,NaN,0.0311337
Unnamed: 3,NaN,2.18432e-16,NaN,-3.81229e-17,NaN,3.70087e-16,NaN,1.02879e-16
Unnamed: 4,NaN,2.22032e-16,NaN,4.76894e-16,NaN,8.16841e-16,NaN,-5.71252e-17
Unnamed: 5,NaN,2.60732e-17,NaN,1.9386e-17,NaN,9.71128e-18,NaN,7.14301e-18
Unnamed: 6,NaN,8.07435e-18,NaN,-1.4001e-16,NaN,-2.04803e-16,NaN,7.81423e-17
Unnamed: 7,NaN,4.42224e-17,NaN,-5.6088e-17,NaN,-3.26818e-17,NaN,1.52996e-16
Unnamed: 8,NaN,8.86776e-17,NaN,-1.76476e-16,NaN,-2.20375e-16,NaN,2.17247e-16
Unnamed: 9,NaN,1.36262e-16,NaN,-2.63726e-16,NaN,-3.27458e-16,NaN,2.766e-16
Unnamed: 10,NaN,1.78275e-16,NaN,-3.24107e-16,NaN,-3.97023e-16,NaN,3.26361e-16
